<a href="https://colab.research.google.com/github/lingyu001/Recommendation/blob/master/YoutubeDNN_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! wget http://files.grouplens.org/datasets/movielens/ml-1m.zip -O ./ml-1m.zip
! wget https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess.py -O preprocess.py
! wget https://raw.githubusercontent.com/lingyu001/Recommendation/master/model_utils.py -O model_utils.py
! unzip -o ml-1m.zip 

--2022-10-04 20:57:23--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘./ml-1m.zip’

./ml-1m.zip         100%[===================>]   5.64M  27.5MB/s    in 0.2s    

2022-10-04 20:57:23 (27.5 MB/s) - ‘./ml-1m.zip’ saved [5917549/5917549]

--2022-10-04 20:57:23--  https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6687 (6.5K) [text/plain]
Saving to: ‘preprocess.py’

preprocess.py       100%[===================>]   6.53K

In [ ]:
# ! pip install -q deepmatch

     |████████████████████████████████| 147 kB 46.3 MB/s 
     |████████████████████████████████| 2.9 MB 61.2 MB/s 


In [15]:
import pandas as pd
from ast import literal_eval
from preprocess import gen_data_set, gen_model_input
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from model_utils import DNN, EmbeddingIndex, NoMask, PoolingLayer, l2_normalize,\
 reduce_mean, SampledSoftmaxLayer, get_item_embedding, sampledsoftmaxloss

In [3]:
%load_ext autoreload
%autoreload 2

# Data

In [ ]:
# data_path = "./"

# unames = ['user_id','gender','age','occupation','zip']
# user = pd.read_csv(data_path+'ml-1m/users.dat',sep='::',header=None,names=unames)
# rnames = ['user_id','movie_id','rating','timestamp']
# ratings = pd.read_csv(data_path+'ml-1m/ratings.dat',sep='::',header=None,names=rnames)
# mnames = ['movie_id','title','genres']
# movies = pd.read_csv(data_path+'ml-1m/movies.dat',sep='::',header=None,names=mnames,encoding="unicode_escape")
# movies['genres'] = list(map(lambda x: x.split('|')[0], movies['genres'].values))

# data = pd.merge(pd.merge(ratings,movies),user).iloc[:10000] # taking 1w
# print(data.shape)
# print(data.user_id.nunique())
# print(data.movie_id.nunique())
# data.head(2)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


(10000, 10)
46
2175


,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation,F,1,10,48067


In [33]:
data = pd.read_csv('sample_data/mik_sales_10042022.csv', index_col = False, 
                   parse_dates = ['trans_date']
                   )
data['most_recent_purchased_sku'] = data['most_recent_purchased_sku'].apply(literal_eval)
data['most_recent_view_sku'] = data['most_recent_view_sku'].apply(literal_eval)
data.shape

(1065196, 13)

In [22]:
data.head(3)

,data_source,user_id,sku_number,qty,trans_date,email_address,geo_zip,most_frequent_device_class_general,total_order_value,total_coupon_value,most_recent_purchased_sku,most_recent_view_sku,category_path
0,ADOBE,3790710821443684383,10603160,1,2022-09-21,tyler_duncan7@yahoo.com,12180,Mobile,NaN,NaN,"[10532465, 10467101, 10603159, 10389616, 10532...","[M10297477, M10279687]",root//Shop Categories//Apparel Crafts//T-Shirt...
1,ADOBE,3790710821443684383,10603160,1,2022-09-21,tyler_duncan7@yahoo.com,12180,Mobile,NaN,NaN,"[10532465, 10467101, 10603159, 10389616, 10532...","[M10297477, M10279687]",root//Shop Categories//Apparel Crafts//T-Shirt...
2,ADOBE,3790710821443684383,10329059,1,2022-09-21,tyler_duncan7@yahoo.com,12180,Mobile,NaN,NaN,"[10532465, 10467101, 10603159, 10389616, 10532...","[M10297477, M10279687]",root//Shop Categories//Apparel Crafts//T-Shirt...


In [23]:
data['most_recent_purchased_sku'][0]

['10532465',
 '10467101',
 '10603159',
 '10389616',
 '10532663',
 '10458309',
 '10307717',
 '10508505']

# Feature Engineering

In [35]:
full_item_set = data['sku_number'].unique()

In [36]:
lbe.fit_transform()

array([13929, 13929,  5513, ..., 10948, 10949, 10949])

In [24]:
data.columns

Index(['data_source', 'user_id', 'sku_number', 'qty', 'trans_date',
       'email_address', 'geo_zip', 'most_frequent_device_class_general',
       'total_order_value', 'total_coupon_value', 'most_recent_purchased_sku',
       'most_recent_view_sku', 'category_path'],
      dtype='object')

In [28]:
sparse_features = ['sku_number', 'category_path',
                   'user_id', 'geo_zip', 'most_frequent_device_class_general']

SEQ_LEN = 20 #50
NUMBER_NEG_SAMPLE = 1 # used for generate 1:1 positive:negative samples


In [32]:
data['sku_number']

0          13930
1          13930
2           5514
3           5514
4          14339
           ...  
1065191    14141
1065192    10949
1065193    10949
1065194    10950
1065195    10950
Name: sku_number, Length: 1065196, dtype: int64

In [29]:
feature_max_idx = {}
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature]) + 1 # add one to all the encoded categories labels
    feature_max_idx[feature] = data[feature].max() + 1

In [30]:
feature_max_idx

{'sku_number': 48749,
 'category_path': 2112,
 'user_id': 185785,
 'geo_zip': 14212,
 'most_frequent_device_class_general': 3}

In [ ]:
data.head(2)

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,1,644,5,978300760,One Flew Over the Cuckoo's Nest (1975),8,1,1,9,18
1,1,385,3,978302109,James and the Giant Peach (1996),3,1,1,9,18


In [31]:
user_profile = data[['user_id', 'geo_zip', 'most_frequent_device_class_general']].drop_duplicates('user_id')
user_profile.set_index("user_id", inplace=True)
item_profile = data[["sku_number"]].drop_duplicates('sku_number')
user_profile.shape, item_profile.shape

((185784, 2), (48748, 1))

In [ ]:
# user_item_list = data.groupby("user_id")['movie_id'].apply(list)
user_item_list = data.groupby("user_id")['movie_id'].apply(list)

user_item_list.head()

user_id
1    [644, 385, 483, 1881, 1306, 646, 731, 1570, 35...
2    [644, 1735, 1283, 1077, 656, 1121, 692, 773, 1...
3    [644, 1570, 487, 647, 357, 693, 909, 670, 72, ...
4    [644, 1881, 1735, 1283, 320, 1545, 152, 1121, ...
5    [644, 1306, 359, 1283, 405, 715, 320, 604, 154...
Name: movie_id, dtype: object

In [ ]:
train_set, test_set = gen_data_set(data, SEQ_LEN, NUMBER_NEG_SAMPLE)

100%|██████████| 46/46 [00:00<00:00, 216.09it/s]

7 8


In [ ]:
# reviewerID,  - user id
# pos_list[i], - movie id
# 1, - label y
# hist[::-1][:seq_len], - most recent watched movie id list before watching this movie, up to 50
# seq_len
# genres_hist[::-1][:seq_len], - most recent watched movie's genres list before watching this movie, up to 50
# genres_list[i], - this movie's genre
# rating_list[i] - this movie's rating
# train_set[12]

In [ ]:
train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)
test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)
train_label.shape, train_model_input['user_id'].shape, test_model_input['user_id'].shape, test_label.shape

/content/preprocess.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_seq_genres = np.array([line[5] for line in train_set])


((19816,), (19816,), (46,), (46,))

In [ ]:
train_model_input

{'user_id': array([35, 10, 36, ..., 35, 34, 40]),
 'movie_id': array([ 606, 1964, 1790, ..., 1593,  394, 1780]),
 'hist_movie_id': array([[1671,  644,  326, ..., 1504,  732,  823],
        [ 759, 1874, 1681, ..., 2044, 1999,  344],
        [ 600, 1155,  553, ...,  350,  226, 1872],
        ...,
        [1076,  627, 2107, ...,  845, 1067, 1919],
        [ 236, 2165, 1458, ...,  544, 1599,  263],
        [ 774,  897,  596, ...,  138,  886,  993]], dtype=int32),
 'hist_genres': array([[ 5,  8,  8, ...,  5,  5,  5],
        [11,  4, 11, ...,  5,  5,  4],
        [ 8,  3,  3, ...,  5,  5,  5],
        ...,
        [ 8,  5, 16, ...,  8,  8,  8],
        [ 1,  5,  1, ...,  4,  2,  5],
        [ 8,  6,  6, ...,  5,  2,  5]], dtype=int32),
 'hist_len': array([50, 50, 50, ..., 50, 50, 50]),
 'genres': array([ 1,  7, 16, ...,  5,  8,  8]),
 'gender': array([2, 2, 2, ..., 2, 2, 1]),
 'age': array([2, 5, 2, ..., 2, 4, 3]),
 'occupation': array([ 5,  4,  3, ...,  5,  8, 17]),
 'zip': array([ 4, 23, 

# Create Embeddings

In [ ]:
from collections import Counter
train_counter = Counter(train_model_input['movie_id'])
item_count = [train_counter.get(i,0) for i in range(feature_max_idx['movie_id'])]
len(train_counter), type(train_counter), len(item_count)

(2175, collections.Counter, 2176)

In [ ]:
import tensorflow as tf
from tensorflow.python.keras.initializers import RandomNormal, Zeros
from tensorflow.python.keras.layers import Embedding, Input, Lambda
from tensorflow.python.keras.regularizers import l2
if tf.__version__ >= '2.0.0':
    tf.compat.v1.disable_eager_execution()
else:
    K.set_learning_phase(True)

### Create Feature Embeddings



In [ ]:
# item features
item_feature_columns = ['movie_id']
# User features
user_feature_columns = ['user_id',
                        "gender",
                        "age",
                        "occupation",
                        "zip",
                        'hist_movie_id',
                        'hist_genres',
                        'hist_len'
                        ]


In [ ]:
feature_max_idx

{'movie_id': 2176,
 'user_id': 47,
 'gender': 3,
 'age': 8,
 'occupation': 18,
 'zip': 47,
 'genres': 19}

In [ ]:
user_sparse_feature_columns = user_feature_columns[0:5]
user_sparse_feature_columns

['user_id', 'gender', 'age', 'occupation', 'zip']

In [ ]:
user_seq_sparse_feature_columns = user_feature_columns[5:-1]
user_seq_sparse_feature_columns 

['hist_movie_id', 'hist_genres']

In [ ]:
# embedding input and output specification
embeddings_initializer = RandomNormal(mean=0.0, stddev=0.0001, seed=2020)
embedding_dim = 32
l2_reg = 1e-6
# use a dict to collect embeddings
sparse_embedding = {}

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
# user embeddings
for name in user_sparse_feature_columns: 
  emb = Embedding(
      input_dim = feature_max_idx[name], 
      output_dim = 16,
      embeddings_initializer=embeddings_initializer,
      embeddings_regularizer=l2(l2_reg),
      name='sparse_' + 'emb_' + name
  )
  emb.trainable = True
  sparse_embedding[name] = emb
for name in user_seq_sparse_feature_columns: 
   emb = Embedding(feature_max_idx[name[5:]], embedding_dim,
                            embeddings_initializer=embeddings_initializer,
                            embeddings_regularizer=l2(
                                l2_reg),
                            name='sparse_' + 'seq_emb_'  + name,
                            mask_zero=False) # True
   emb.trainable = True
   sparse_embedding[name] = emb

In [ ]:
# item embeddings
item_feature_name = item_feature_columns[0] # only include item id 
emb = Embedding(
      input_dim = feature_max_idx[item_feature_name], 
      output_dim = embedding_dim,
      embeddings_initializer=embeddings_initializer,
      embeddings_regularizer=l2(l2_reg),
      name='sparse_' + 'emb_' + item_feature_name
  )
sparse_embedding[item_feature_name] = emb

In [ ]:
sparse_embedding # include both user and item embedding layers

{'user_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f87aefcc050>,
 'gender': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f87aefc6e90>,
 'age': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f87aefc62d0>,
 'occupation': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f87aefc61d0>,
 'zip': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f87aefc4790>,
 'hist_movie_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f87aefccd90>,
 'hist_genres': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f87aefc4350>,
 'movie_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f87aefcc350>}

# Create User Inputs


## Create Inputs Tensor

In [ ]:
# user features inputs
from collections import OrderedDict
user_features = OrderedDict()
for name in user_sparse_feature_columns:
  user_features[name] = Input(
                shape=(1,), 
                name=name, 
                dtype="int32"
                )
for name in user_seq_sparse_feature_columns:
  user_features[name] = Input(
                shape=(SEQ_LEN,), 
                name=name, 
                dtype="int32"
                )
length_name = 'hist_len'
if length_name in user_feature_columns:
  user_features[length_name] = Input((1,), name=length_name, dtype='int32')

In [ ]:
user_inputs_list = list(user_features.values())
user_inputs_list

[<tf.Tensor 'user_id:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'gender:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'age:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'occupation:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'zip:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'hist_movie_id:0' shape=(None, 50) dtype=int32>,
 <tf.Tensor 'hist_genres:0' shape=(None, 50) dtype=int32>,
 <tf.Tensor 'hist_len:0' shape=(None, 1) dtype=int32>]

In [ ]:
# item feature inputs
item_features = OrderedDict()
for name in item_feature_columns:
  item_features[name] = Input(
                shape=(1,), 
                name=name, 
                dtype="int32"
                )
item_inputs_list = list(item_features.values())


In [ ]:
item_inputs_list

[<tf.Tensor 'movie_id:0' shape=(None, 1) dtype=int32>]

## Embed Inputs

In [ ]:
# Embed sparse inputs
from collections import defaultdict
user_sparse_embeded_input = defaultdict(list)

for name in user_sparse_feature_columns:
  user_sparse_embeded_input[name] = sparse_embedding[name](user_features[name])

In [ ]:
# Embed the sequence sparse inputs, need to add pooling layer to it
seq_embeded_dict = defaultdict(list)

for name in user_seq_sparse_feature_columns:
  seq_embeded_dict[name] = sparse_embedding[name](user_features[name])

# from deepctr.layers.sequence import SequencePoolingLayer
# for name in user_seq_sparse_feature_columns:
#   user_seq_sparse_embeded_input[name] = SequencePoolingLayer('mean', supports_masking=False)([seq_embeded_dict[name], user_features[length_name]])

from tensorflow.keras.layers import AveragePooling1D
user_seq_sparse_embeded_input = defaultdict(list)
for name in user_seq_sparse_feature_columns:
  user_seq_sparse_embeded_input[name] = AveragePooling1D(pool_size=50, padding='valid')(seq_embeded_dict[name])

In [ ]:
# vec = AveragePooling1D(pool_size=50, strides = 50, padding='valid')
# vec(seq_embeded_dict[name])

In [ ]:
# from deepctr.layers.sequence import SequencePoolingLayer
# vec2 = SequencePoolingLayer('mean', supports_masking=False)
# vec2([seq_embeded_dict[name], 35])

In [ ]:
# combine embeded inputs into a list
user_sparse_embedding_list = list(user_sparse_embeded_input.values()) + list(user_seq_sparse_embeded_input.values())
user_sparse_embedding_list

[<tf.Tensor 'sparse_emb_user_id/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_gender/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_age/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_occupation/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_zip/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'average_pooling1d/Squeeze:0' shape=(None, 1, 32) dtype=float32>,
 <tf.Tensor 'average_pooling1d_1/Squeeze:0' shape=(None, 1, 32) dtype=float32>]

## Combine Embeded Input (User only)

In [ ]:
from tensorflow.python.keras.layers import Flatten, Concatenate, Layer, Add

In [ ]:
user_dnn_input = Flatten()(Concatenate(axis=2)(user_sparse_embedding_list))
user_dnn_input

<tf.Tensor 'flatten/Reshape:0' shape=(None, 144) dtype=float32>

# Sepecify Model Layers

### User Layers

In [ ]:
from tensorflow.keras import layers

In [ ]:
user_dnn_hidden_units = (128,64, embedding_dim)
dnn_activation='relu'
dnn_use_bn=False
l2_reg_dnn=0
l2_reg_embedding=1e-6
dnn_dropout=0
output_activation='linear'
temperature=0.05,
# sampler_config=sampler_config
seed=1024

In [ ]:
from deepctr.layers import DNN

In [ ]:
dnn_layers = DNN(hidden_units = user_dnn_hidden_units, 
                  activation='relu', 
                  l2_reg=0, 
                  dropout_rate=0, 
                  use_bn=False, 
                  output_activation='linear',
                  seed=1024
                  )



In [ ]:
user_dnn_output = dnn_layers(user_dnn_input)
user_dnn_output = l2_normalize(user_dnn_output)
user_dnn_output

<tf.Tensor 'lambda/l2_normalize:0' shape=(None, 32) dtype=float32>

### Item Layer

In [ ]:
# Create item index tensor
item_vocabulary_size = feature_max_idx[item_feature_name]
item_index = EmbeddingIndex(list(range(item_vocabulary_size)))(item_features[item_feature_name])

# embed the item index input
item_embeding = sparse_embedding[item_feature_name]
item_embedding_weight = NoMask()(item_embeding(item_index))
pooling_item_embedding_weight = PoolingLayer()([item_embedding_weight])
pooling_item_embedding_weight = l2_normalize(pooling_item_embedding_weight)
pooling_item_embedding_weight


<tf.Tensor 'lambda_1/l2_normalize:0' shape=(2176, 32) dtype=float32>

In [ ]:
pooling_item_embedding_weight.eval(session=K.get_session()).shape

(2176, 32)

In [ ]:
# .eval(session=K.get_session())

### Output Layer

In [ ]:
output = SampledSoftmaxLayer()(
    [pooling_item_embedding_weight, user_dnn_output, item_features[item_feature_name]])
output

<tf.Tensor 'sampled_softmax_layer/ExpandDims:0' shape=(None, 1) dtype=float32>

In [ ]:
# from model_utils import SampledSoftmaxLayer
# sf = SampledSoftmaxLayer()

In [ ]:
# from tensorflow.keras.layers import Softmax
# sf2 = Softmax()
# import numpy as np
# x = np.asarray([[1., 2., 1.]])
# x
# sf2(x).eval(session=K.get_session())

In [ ]:
pooling_item_embedding_weight

<tf.Tensor 'lambda_1/l2_normalize:0' shape=(2176, 32) dtype=float32>

In [ ]:
user_dnn_output

<tf.Tensor 'lambda/l2_normalize:0' shape=(None, 32) dtype=float32>

In [ ]:
item_features[item_feature_name]

<tf.Tensor 'movie_id:0' shape=(None, 1) dtype=int32>

## Build Model

In [ ]:
from tensorflow.python.keras.models import Model

In [ ]:
model = Model(inputs=user_inputs_list + item_inputs_list, outputs=output)

model.__setattr__("user_input", user_inputs_list)
model.__setattr__("user_embedding", user_dnn_output)

model.__setattr__("item_input", item_inputs_list)
model.__setattr__("item_embedding",
                  get_item_embedding(pooling_item_embedding_weight, item_features[item_feature_name]))


# Model Compile and Training

In [ ]:
model.compile(optimizer="adam", loss=sampledsoftmaxloss)

In [ ]:
history = model.fit(train_model_input, train_label,
                    batch_size=512, epochs=20, verbose=1, validation_split=0.0, )

Train on 19816 samples
Epoch 1/20
19816/19816 [==============================] - 2s 97us/sample - loss: 5.1630
Epoch 2/20
19816/19816 [==============================] - 1s 55us/sample - loss: 5.0705
Epoch 3/20
19816/19816 [==============================] - 1s 52us/sample - loss: 5.0703
Epoch 4/20
19816/19816 [==============================] - 1s 45us/sample - loss: 5.0599
Epoch 5/20
19816/19816 [==============================] - 1s 28us/sample - loss: 5.0527
Epoch 6/20
19816/19816 [==============================] - 1s 28us/sample - loss: 5.0114
Epoch 7/20
19816/19816 [==============================] - 1s 27us/sample - loss: 5.0036
Epoch 8/20
19816/19816 [==============================] - 1s 28us/sample - loss: 4.9630
Epoch 9/20
19816/19816 [==============================] - 1s 27us/sample - loss: 4.9372
Epoch 10/20
19816/19816 [==============================] - 1s 27us/sample - loss: 4.9121
Epoch 11/20
19816/19816 [==============================] - 1s 28us/sample - loss: 4.8994
Epoch 1

In [ ]:
res = model.predict(train_model_input)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2458: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


In [ ]:
res

array([[5.054029 ],
       [3.8712769],
       [4.1115036],
       ...,
       [4.192506 ],
       [5.27453  ],
       [3.5851386]], dtype=float32)

In [ ]:
test_model_input['user_id'].shape

(46,)

In [ ]:
train_model_input

{'user_id': array([35, 10, 36, ..., 35, 34, 40]),
 'movie_id': array([ 606, 1964, 1790, ..., 1593,  394, 1780]),
 'hist_movie_id': array([[1671,  644,  326, ..., 1504,  732,  823],
        [ 759, 1874, 1681, ..., 2044, 1999,  344],
        [ 600, 1155,  553, ...,  350,  226, 1872],
        ...,
        [1076,  627, 2107, ...,  845, 1067, 1919],
        [ 236, 2165, 1458, ...,  544, 1599,  263],
        [ 774,  897,  596, ...,  138,  886,  993]], dtype=int32),
 'hist_genres': array([[ 5,  8,  8, ...,  5,  5,  5],
        [11,  4, 11, ...,  5,  5,  4],
        [ 8,  3,  3, ...,  5,  5,  5],
        ...,
        [ 8,  5, 16, ...,  8,  8,  8],
        [ 1,  5,  1, ...,  4,  2,  5],
        [ 8,  6,  6, ...,  5,  2,  5]], dtype=int32),
 'hist_len': array([50, 50, 50, ..., 50, 50, 50]),
 'genres': array([ 1,  7, 16, ...,  5,  8,  8]),
 'gender': array([2, 2, 2, ..., 2, 2, 1]),
 'age': array([2, 5, 2, ..., 2, 4, 3]),
 'occupation': array([ 5,  4,  3, ...,  5,  8, 17]),
 'zip': array([ 4, 23, 

# Predict: Generate Item and User Embeddings

In [ ]:
# 4. Generate user features for testing and full item features for retrieval
test_user_model_input = test_model_input
all_item_model_input = {"movie_id": item_profile['movie_id'].values,}

user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)

user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
# user_embs = user_embs[:, i, :]  # i in [0,k_max) if MIND
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

print(user_embs.shape)
print(item_embs.shape)

(46, 32)
(2175, 32)


array([[ 0.1762459 , -0.00250923,  0.15487695, ...,  0.12628743,
        -0.35957137, -0.00677316],
       [-0.32162988,  0.19727159,  0.15648587, ...,  0.06774145,
         0.03004007,  0.00682755],
       [-0.14978446,  0.27408543,  0.02244441, ..., -0.13048121,
         0.13271852,  0.10681918],
       ...,
       [-0.21950896,  0.07887422,  0.19160384, ...,  0.1565665 ,
        -0.09389586,  0.17347859],
       [ 0.10354648,  0.22348611, -0.03165336, ..., -0.14569618,
         0.33465326,  0.26777056],
       [ 0.27626848,  0.16945344,  0.16872141, ..., -0.00689157,
         0.02326779, -0.10236465]], dtype=float32)

In [ ]:
import numpy as np
a = np.array([ 0.11175336,  0.4548011 , -0.15772209,  0.3445765 , -0.0619677 ,
        0.06464498, -0.10026787, -0.07865412, -0.05304453,  0.19952899,
        0.14534044,  0.16223468,  0.02545189, -0.24614649,  0.02315353,
        0.09893572, -0.07596418, -0.09769203,  0.02297432,  0.1150127 ,
       -0.09194405,  0.05969758, -0.01262948, -0.04111078, -0.52638304,
       -0.01993592, -0.11702564, -0.13895226, -0.04519685, -0.30629617,
       -0.02717471, -0.03397018])
b = np.array([-0.22946945,  0.31338242,  0.17805392, -0.40977573,  0.06601848,
       -0.32935393,  0.11766144, -0.08969792, -0.01543093,  0.00668303,
       -0.1148053 ,  0.02085993,  0.3859076 , -0.07861419,  0.00837284,
       -0.01701969,  0.3163019 , -0.36242014, -0.04939312, -0.02236807,
       -0.12954137,  0.1923324 ,  0.04133848,  0.03154316, -0.00369814,
        0.15590565, -0.05733275,  0.0741621 ,  0.05379602,  0.07140778,
       -0.11147177, -0.03305427])

In [ ]:
a.dot(b)

-0.06165534518726169